In [9]:
import pandas as pd
import numpy as np

In [1]:
df = pd.read_csv("lienminh.csv")
df.head()

,killsDiff,minionsKilledDiff,wardPlacedDiff,firstBlood,heralds,dragons,teamWins
0,3,-2,13,blue,none,none,red
1,0,-66,0,red,red,red,red
2,-4,-17,0,red,none,blue,red
3,-1,-34,28,red,blue,none,red
4,0,-15,58,red,none,red,red


In [2]:
killsDiff = df['killsDiff'].values
minionsKilledDiff = df['minionsKilledDiff'].values
wardPlacedDiff = df['wardPlacedDiff'].values
firstBlood = df['firstBlood'].values
heralds = df['heralds'].values
dragons = df['dragons'].values

teamWins = df['teamWins'].values

## Bài tập

Hãy xây dựng mô hình Naive Bayes sử dụng tất các cả thông tin mà dữ liệu cho:
- killsDiff, minionsKilledDiff, wardPlacedDiff: xấp xỉ bằng phân mối normal
- firstBlood, heralds, dragons: xấp xỉ bằng phân phối category

Note: kết quả không nhất thiết tốt hơn, quan trọng phải đúng



In [30]:
## xác suất tiên nghiệm
P_blueWins = sum(teamWins == 'blue') / len(df)
P_redWins = sum(teamWins == 'red') / len(df)

### Guassian

In [29]:
def normal(x, mean, std):
    return 1/(np.sqrt(2*np.pi)*std)*np.exp(-np.square(x-mean)/(2*np.square(std)))

In [32]:
## tính tham số killsDiff
killsDiff_mean_blueWins = np.mean(killsDiff[teamWins == 'blue'])
killsDiff_std_blueWins = np.std(killsDiff[teamWins == 'blue'])
P_killsDiff_blueWins = normal(killsDiff, killsDiff_mean_blueWins, killsDiff_std_blueWins)

killsDiff_mean_redWins = np.mean(killsDiff[teamWins == 'red'])
killsDiff_std_redWins = np.std(killsDiff[teamWins == 'red'])
P_killsDiff_redWins = normal(killsDiff, killsDiff_mean_redWins, killsDiff_std_redWins)

## tính tham số minionsKilledDiff
minionsKilledDiff_mean_blueWins = np.mean(minionsKilledDiff[teamWins == 'blue'])
minionsKilledDiff_std_blueWins = np.std(minionsKilledDiff[teamWins == 'blue'])
P_minionsKilledDiff_blueWins = normal(minionsKilledDiff, minionsKilledDiff_mean_blueWins, minionsKilledDiff_std_blueWins)

minionsKilledDiff_mean_redWins = np.mean(minionsKilledDiff[teamWins == 'red'])
minionsKilledDiff_std_redWins = np.std(minionsKilledDiff[teamWins == 'red'])
P_minionsKilledDiff_redWins = normal(minionsKilledDiff, minionsKilledDiff_mean_redWins, minionsKilledDiff_std_redWins)

## tính tham số wardPlacedDiff
wardPlacedDiff_mean_blueWins = np.mean(wardPlacedDiff[teamWins == 'blue'])
wardPlacedDiff_std_blueWins = np.std(wardPlacedDiff[teamWins == 'blue'])
P_wardPlacedDiff_blueWins = normal(wardPlacedDiff, wardPlacedDiff_mean_blueWins, wardPlacedDiff_std_blueWins)

wardPlacedDiff_mean_redWins = np.mean(wardPlacedDiff[teamWins == 'red'])
wardPlacedDiff_std_redWins = np.std(wardPlacedDiff[teamWins == 'red'])
P_wardPlacedDiff_redWins = normal(wardPlacedDiff, wardPlacedDiff_mean_redWins, wardPlacedDiff_std_redWins)

In [39]:
## P(B|kills, minions, wards)
P_blueWins_guassian = P_blueWins * P_killsDiff_blueWins * P_minionsKilledDiff_blueWins * P_wardPlacedDiff_blueWins
## P(R|kills, minions, wards)
P_redWins_guassian = P_redWins * P_killsDiff_redWins * P_minionsKilledDiff_redWins * P_wardPlacedDiff_redWins

## Accuracy
np.mean((P_blueWins_guassian > P_redWins_guassian) == (teamWins == 'blue'))

0.7082700678206296

In [51]:
X_train_gaussian = df[['killsDiff', 'minionsKilledDiff', 'wardPlacedDiff']].values

## Dùng sklearn
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB().fit(X_train_gaussian, teamWins)
np.mean(teamWins == nb.predict(X_train_gaussian))

0.7082700678206296

### Categorical

In [57]:
is_bluefirstBlood = (firstBlood == 'blue').astype(float)

## tính P(blueFistBlood|B)
P_blueFirstBlood_blueWins  = np.sum(firstBlood[teamWins == 'blue'] == 'blue') / np.sum(teamWins == 'blue')
P_blueFirstBlood_blueWins = P_blueFirstBlood_blueWins * is_bluefirstBlood + (1-P_blueFirstBlood_blueWins)*(1-is_bluefirstBlood)

## tính P(blueFirstBlood|R)
P_blueFirstBlood_redWins = np.sum(firstBlood[teamWins == 'red'] == 'blue') / np.sum(teamWins == 'red')
P_blueFirstBlood_redWins = P_blueFirstBlood_redWins * is_bluefirstBlood + (1-P_blueFirstBlood_redWins)*(1-is_bluefirstBlood)

In [58]:
is_blueHeralds = (heralds == 'blue').astype(float)

## tính P(blueHeralds|B)
P_blueHeralds_blueWins  = np.sum(heralds[teamWins == 'blue'] == 'blue') / np.sum(teamWins == 'blue')
P_blueHeralds_blueWins = P_blueHeralds_blueWins * is_blueHeralds + (1-P_blueHeralds_blueWins)*(1-is_blueHeralds)

## tính P(blueHeralds|R)
P_blueHeralds_redWins = np.sum(heralds[teamWins == 'red'] == 'blue') / np.sum(teamWins == 'red')
P_blueHeralds_redWins = P_blueHeralds_redWins * is_blueHeralds + (1-P_blueHeralds_redWins)*(1-is_blueHeralds)

In [59]:
is_blueDragons = (dragons == 'blue').astype(float)

## tính P(blueDragons|B)
P_blueDragons_blueWins  = np.sum(dragons[teamWins == 'blue'] == 'blue') / np.sum(teamWins == 'blue')
P_blueDragons_blueWins = P_blueDragons_blueWins * is_blueDragons + (1-P_blueDragons_blueWins)*(1-is_blueDragons)

## tính P(blueDragons|R)
P_blueDragons_redWins = np.sum(dragons[teamWins == 'red'] == 'blue') / np.sum(teamWins == 'red')
P_blueDragons_redWins = P_blueDragons_redWins * is_blueDragons + (1-P_blueDragons_redWins)*(1-is_blueDragons)

In [61]:
## P(B|first blood, heralds, dragons)
P_blueWins_categorical = P_blueWins * P_blueFirstBlood_blueWins * P_blueHeralds_blueWins * P_blueDragons_blueWins
## P(R|first blood, heralds, dragons)
P_redWins_categorical = P_redWins * P_blueFirstBlood_redWins * P_blueHeralds_redWins * P_blueDragons_redWins

## Accuracy
np.mean((P_blueWins_categorical > P_redWins_categorical) == (teamWins == 'blue'))

0.6133211863548942

In [50]:
X_train_categorical = (df[['firstBlood', 'heralds', 'dragons']].values == 'blue').astype(float)

## Dùng sklearn
from sklearn.naive_bayes import CategoricalNB
nb = CategoricalNB(alpha=1.0e-10).fit(X_train_categorical, teamWins)
np.mean(teamWins == nb.predict(X_train_categorical))

0.6133211863548942